# Project: Traffic Simulation Model


## Introduction

This project implements a traffic simulation model using the Mesa framework. The simulation aims to model the movement of cars within a city grid, incorporating traffic lights and parking spots. The primary goal is to simulate realistic traffic behavior, including car movements, traffic light interactions, and parking dynamics.

The simulation is built on a 24x24 grid, where each cell can represent different elements such as roads, buildings, parking spots, and traffic lights. Car agents navigate through the grid based on predefined directional coordinates and traffic light states, aiming to reach their target parking spots.

Key features of the project include:

- Car Agents: Simulated cars that move around the grid, following traffic rules and aiming to park in available spots.
- Traffic Light Agents: Traffic lights that control the flow of car agents at intersections, switching between red and green states.
- Grid Representation: A multi-layer grid that includes buildings, roads, and parking spots.
- Visualization: Tools for visualizing the simulation, including the positions of car agents and the states of traffic lights.

The project leverages the Mesa framework for agent-based modeling and includes various components for data collection and visualization to analyze the simulation results.

## Installation



To set up the project, you need to install several dependencies. These libraries are essential for running the traffic simulation model and visualizing the results.

### Required Libraries

- **Mesa (version 3.1.0.dev)**: A Python framework for agent-based modeling. It provides the core functionality for creating and managing agents, as well as tools for visualization and data collection.
- **Seaborn**: A data visualization library based on Matplotlib. It is used for creating informative and attractive statistical graphics.
- **Solara**: A library for building interactive web applications in Python. It is used for creating interactive visualizations of the simulation.
- **Pandas**: A powerful data manipulation and analysis library. It is used for handling and analyzing data within the simulation.
- **NumPy**: A fundamental package for scientific computing with Python. It is used for numerical operations and handling arrays.
- **Random**: A module for generating random numbers. It is used for randomizing agent behaviors and movements.
- **Time**: A module for time-related functions. It is used for managing simulation timing and delays.

### Installation Steps

To install the required libraries, you can use the following commands:
```python
#%pip install --quiet --upgrade mesa==3.1.0.dev
#%pip install solara
#%pip install seaborn
#%pip install pandas
#%pip install numpy

In [1]:

import time
# Data visualization tools.
import seaborn as sns
import random
import numpy as np

# Data manipulation and analysis. 
import pandas as pd

#Mesa library
import mesa
from mesa.visualization.utils import update_counter
from mesa.visualization import SolaraViz, make_plot_component, make_space_component
from mesa.datacollection import DataCollector
print(f"Mesa version: {mesa.__version__}")


Mesa version: 3.0.3


## Map Construction



In this section, we will define the structure and layout of the simulation environment. The map construction involves setting up a grid that represents the city, including buildings, parking spots, directional coordinates, and traffic lights. Each of these elements plays a crucial role in the simulation:

- **Buildings**: Represent static structures within the city grid. They serve as obstacles that car agents must navigate around.
- **Parking Spots**: Designated areas where car agents aim to park. These spots are the primary destinations for the car agents.
- **Directional Coordinates**: Predefined paths that guide the movement of car agents through the grid. These coordinates help simulate realistic traffic flow.
- **Traffic Lights**: Control the flow of car agents at intersections. They switch between red and green states to manage traffic and prevent collisions.

We will use the Mesa framework to create and manage the grid, ensuring that all elements are correctly placed and interact as expected. The map construction is a crucial step in creating a realistic and functional traffic simulation model.

#### Buildings

In [2]:
from mapBuild.buildings import buildings_coords

#### Parking spots

In [3]:
from mapBuild.parkingSpots import parking_spots

#### Direccional coords(Left, Right, Down, Up)

In [4]:
from mapBuild.leftCoords import left_coords
from mapBuild.rightCoords import right_coords
from mapBuild.upCoords import up_coords
from mapBuild.downCoords import down_coords

##### Secondary directional coord(DownLeft, DownRight, UpLeft, UpRight)

In [5]:
from mapBuild.downLeftCoords import down_left_coords
from mapBuild.downRightCoords import down_right_coords
from mapBuild.upLeftCoords import up_left_coords
from mapBuild.upRightCoords import up_right_coords

In [6]:
# Storing the coordinates in a dictionary
coords = {
    "left_coords": left_coords,
    "right_coords": right_coords,
    "up_coords": up_coords,
    "down_coords": down_coords,
    "down_left_coords": down_left_coords,
    "down_right_coords": down_right_coords,
    "up_left_coords": up_left_coords,
    "up_right_coords": up_right_coords,
}

#### Traffic Lights

In [7]:
from mapBuild.trafficLights import traffic_light_coords

## Traffic Model

In this section, we will explain the traffic simulation model, its components, and main functionalities. The model is designed to simulate the movement of car agents within a city grid, incorporating traffic lights and parking spots to create a realistic traffic environment.

### Components

1. **Car Agents**:
   - **Purpose**: Simulate individual cars moving through the city grid.
   - **Behavior**: Car agents navigate the grid based on directional coordinates, obey traffic lights, and aim to park in available parking spots.

2. **Traffic Light Agents**:
   - **Purpose**: Control the flow of car agents at intersections.
   - **Behavior**: Traffic lights switch between red and green states at regular intervals, managing the movement of car agents and preventing collisions.

3. **Grid**:
   - **Purpose**: Represent the city layout, including roads, buildings, parking spots, and traffic lights.
   - **Structure**: A 24x24 grid where each cell can represent different elements such as roads, buildings, parking spots, and traffic lights.

### Main Functionalities

1. **Initialization**:
   - The model initializes the grid, placing buildings, parking spots, directional coordinates, and traffic lights in their respective positions.
   - Car agents are created and placed on the grid, ready to navigate the city.

2. **Agent Movement**:
   - Car agents move through the grid based on predefined directional coordinates.
   - They obey traffic lights and navigate around buildings to reach their target parking spots.

3. **Traffic Light Control**:
   - Traffic light agents manage the flow of car agents at intersections.
   - They switch between red and green states at regular intervals, ensuring smooth traffic flow and preventing collisions.

4. **Parking Dynamics**:
   - Car agents aim to park in available parking spots.
   - The model tracks the occupancy of parking spots and updates the status of car agents accordingly.

This comprehensive model allows for the simulation of realistic traffic behavior, providing valuable insights into traffic dynamics and potential improvements for urban planning.

In [8]:
class TrafficModel(mesa.Model):
    def __init__(
        self,
        width=24,
        height=24,
        num_agents=1,
        coords=None,
        buildings_coords=None,
        parking_coords=None,
        traffic_light_coords=None,
    ):
        super().__init__()
        #ATTRIBUTES OF THE MODEL----------------------------------------------------------------------------------
        self.width = width
        self.height = height
        self.num_agents = num_agents
        self.coords = coords
        self.buildings_coords = buildings_coords
        self.parkings_coords = parking_coords
        self.traffic_light_coords = traffic_light_coords

        # Initialize random generator and step counter
        self.random = random.Random()
        self.steps = 0

        # Initialize direction dictionary and global maps
        self.directions = {}
        self.global_map_cars = {}
        self.global_map_traffic_lights = {}
        self.traffic_light_groups = {}

        # Create a dictionary mapping each parking spot to a unique key
        self.ParkingSpots = {i + 1: spot for i, spot in enumerate(self.parkings_coords)}

        # Initialize grid layers
        self.initialize_layers()

        #CREATION OF AGENTS IN THE GRID---------------------------------------    
        # Initialize agents and their positions
        self.initialize_agents()

        # Initialize data collector
        self.datacollector = mesa.DataCollector(
            model_reporters={},
            agent_reporters={}
        )
        self.datacollector.collect(self)
        
    #LAYERS OF THE GRID---------------------------------------
    def initialize_layers(self):
        """Initialize the grid layers for buildings, parking spots, and traffic monitoring."""
        buildingLayer = mesa.space.PropertyLayer("building", self.width, self.height, np.int64(0), np.int64)
        parkingsLayer = mesa.space.PropertyLayer("parking", self.width, self.height, np.int64(0), np.int64)
        trafficMonitoringLayer = mesa.space.PropertyLayer("traffic_monitoring", self.width, self.height, np.int64(0), np.int64)

        self.set_building_cells(buildingLayer)
        self.set_parking_cells(parkingsLayer)
        self.set_traffic_monitoring_cells(trafficMonitoringLayer)

        self.grid = mesa.space.MultiGrid(self.width, self.height, True, (buildingLayer, parkingsLayer, trafficMonitoringLayer))
        """I am checking if the traffic lights are checking the right amount of cars in their monitored area
        list_of_cords_of_traffic_lights = [(2, 4),(0, 6)]
        for cord in list_of_cords_of_traffic_lights:
            for agent in self.grid.get_cell_list_contents(cord):
                if isinstance(agent, TrafficLightAgent):
                    print(f"Agent:{agent.traffic_light_id} Found: {agent.cars_in_monitored_area()} in his area.")
        """
       
    #INITIALIZER METHODS----------------------------------------------------------------------------------
    def initialize_agents(self):
        """Initialize the agents in the grid."""
        self.initialize_directions(self.coords)
        self.create_traffic_light_groups()
        self.create_CarAgents()
        self.create_CarAgents_no_target()
        self.place_TrafficLight_agents()

    # Initialize allowed directions for each cell in the grid
    def initialize_directions(self, coords):
        for x in range(self.width):
            for y in range(self.height):
                self.directions[(x, y)] = {
                    "left": False,
                    "right": False,
                    "up": False,
                    "down": False,
                    "down_left": False,
                    "down_right": False,
                    "up_left": False,
                    "up_right": False,
                }

        # Set specific directions for each list
        for coord in coords["left_coords"]:
            if coord in self.directions:
                self.directions[coord]["left"] = True

        for coord in coords["right_coords"]:
            if coord in self.directions:
                self.directions[coord]["right"] = True

        for coord in coords["up_coords"]:
            if coord in self.directions:
                self.directions[coord]["up"] = True

        for coord in coords["down_coords"]:
            if coord in self.directions:
                self.directions[coord]["down"] = True

        for coord in coords["down_left_coords"]:
            if coord in self.directions:
                self.directions[coord]["down_left"] = True

        for coord in coords["down_right_coords"]:
            if coord in self.directions:
                self.directions[coord]["down_right"] = True

        for coord in coords["up_left_coords"]:
            if coord in self.directions:
                self.directions[coord]["up_left"] = True

        for coord in coords["up_right_coords"]:
            if coord in self.directions:
                self.directions[coord]["up_right"] = True

    def create_CarAgents_no_target(self):
        """Create car agents without a target parking spot."""
        number_of_cars = 50
        all_coords = [(x, y) for x in range(self.width) for y in range(self.height)]
        available_coords = [coord for coord in all_coords if coord not in self.parkings_coords and coord not in self.buildings_coords]

        if len(available_coords) < number_of_cars:
            raise ValueError("Not enough available coordinates to place all cars.")

        for _ in range(number_of_cars):
            spawn_position = random.choice(available_coords)
            available_coords.remove(spawn_position)
            agent = CarAgent(self, spawn_position, None)
            self.grid.place_agent(agent, spawn_position)
            

    # Create agents and place them on the grid
    def create_CarAgents(self):
        """Create car agents and place them on the grid."""
        used_parking_spots = set()

        for _ in range(self.num_agents):
            available_spawn_spots = [spot for spot in self.ParkingSpots.values() if spot not in used_parking_spots]

            if not available_spawn_spots:
                print("No available spots for spawn")
                break

            spawn = random.choice(available_spawn_spots)
            used_parking_spots.add(spawn)

            available_target_spots = [spot for spot in self.ParkingSpots.values() if spot != spawn and spot not in used_parking_spots]

            if not available_target_spots:
                print("No available spots for target parking")
                break

            target_parking_spot = random.choice(available_target_spots)
            used_parking_spots.add(target_parking_spot)

            print(f"Spawn: ({spawn}), Target: ({target_parking_spot})")

            agent = CarAgent(self, spawn, target_parking_spot)
            self.grid.place_agent(agent, spawn)
            

    #TRAFFIC LIGHT METHODS----------------------------------------------------------------------------------
    def place_TrafficLight_agents(self):
        for traffic_light_id, traffic_light_info in self.traffic_light_coords.items():
            positions = traffic_light_info["position"]
            initial_state = traffic_light_info["state"]
            group = traffic_light_info["group"]
            monitored_coords = traffic_light_info["monitored_coords"]
            
            # Create separate agents with the same id and place them in the corresponding positions
            for pos in positions:
                sema_agent = TrafficLightAgent(
                    traffic_light_id=traffic_light_id,
                    state=initial_state,
                    model=self,
                    group=group,
                    position=positions,
                    monitored_positions=monitored_coords
                )  # Assign the same id
                self.grid.place_agent(sema_agent, pos)
    
    def create_traffic_light_groups(self):
        """Create a map of traffic light groups."""
        for traffic_light_id, traffic_light_info in self.traffic_light_coords.items():
            group = traffic_light_info["group"]
            if group not in self.traffic_light_groups:
                self.traffic_light_groups[group] = []
            self.traffic_light_groups[group].append(traffic_light_id)

    #GETTER METHODS----------------------------------------------------------------------------------
    def get_cell_directions(self, pos):
        """Fetch the direction info for a specific cell."""
        return self.directions.get(pos, None)

    # Create a global map of the current state of the simulation
    def get_global_map(self):
        """Create a global map of the current state of the simulation."""
        self.global_map_cars = {"Cars": []}
        self.global_map_traffic_lights = {"Traffic_Lights": {}}

        car_agents = []
        trafficLight = {}

        for contents, (x, y) in self.grid.coord_iter():
            for agent in contents:
                if isinstance(agent, CarAgent):
                    car_agents.append(agent)
                elif isinstance(agent, TrafficLightAgent):
                    trafficLight[agent.unique_id] = agent.state == "green"

        car_agents.sort(key=lambda agent: agent.unique_id)

        for agent in car_agents:
            self.global_map_cars["Cars"].append(agent.pos)

        self.global_map_traffic_lights["Traffic_Lights"] = trafficLight

        return self.global_map_cars, self.global_map_traffic_lights
    
        
    # LAYER METHODS----------------------------------------------------------------------------------
    def set_building_cells(self, buildingLayer):
        """Set the value of cells to indicate buildings."""
        for coord in self.buildings_coords:
            buildingLayer.set_cell(coord, 1)

    def set_parking_cells(self, parkingsLayer):
        """Set the value of cells to indicate parking spots."""
        for coord in self.parkings_coords:
            parkingsLayer.set_cell(coord, 1)

    def set_traffic_monitoring_cells(self, trafficMonitoringLayer):
        """Set the value of cells to indicate traffic monitoring spots."""
        all_monitored_coords = []
        for traffic_light in self.traffic_light_coords.values():
            all_monitored_coords.extend(traffic_light["monitored_coords"])
        for coord in all_monitored_coords:
            trafficMonitoringLayer.set_cell(coord, 1)

    # STEP METHOD----------------------------------------------------------------------------------
    # Execute one step of the model, shuffle agents, and collect data
    def step(self):
        # Shuffle and execute the step method for all agents
        self.agents.shuffle_do("step")
        # Collect data for the current step
        self.datacollector.collect(self)
        # Create a global map of the current state
        self.get_global_map()


## Car Agent
Main points of the agent's functionality:

- Initialization:
Each agent has a unique ID and is created with the model. 
It is randomly positioned on the model's grid.
- Movement:
The main movement method is `move()`.
This method is called at each model step `step()`.
- Directions:
A dictionary of directions is used: "up," "down," "left," "right".
- Choosing a random direction:
A random direction is selected from those allowed in the current cell.
- Checking for an empty space:
Before moving, it checks if the new space is empty.

**Basic Configuration:**
- Defines the width and height of the grid (default is 24x24). 
Sets the number of agents (default is 10).
- Path Customization:
Allows defining custom paths using lists of coordinates (x, y) for each direction (up, down, left, right).
- Model Initialization: 
Creates a two-dimensional grid (MultiGrid). 
Initializes the scheduler to manage model steps.
- Agent Creation: 
Randomly places agents on the grid.
- Direction Initialization: 
Uses the initialize_directions() method to set up allowed directions in each cell based on the provided lists.
- Model Steps: The step() method is called at each step to update the state of all agents.

In [9]:
class CarAgent(mesa.Agent):
    def __init__(self, model, spawn_position, target_parking_spot = None):
        super().__init__(model)
        self.active = True
        self.parking_spots = [coord for coord in model.parkings_coords if coord != spawn_position]
        self.distance_travelled = 0
        self.target_parking_spot = target_parking_spot

    def check_semaphore(self, current_position): #for future implementation, check if the agent is a car
        semaphore_agent = self.get_semaphore_agent(current_position)
        if semaphore_agent is None:
            return True
        return self.is_semaphore_green(semaphore_agent, current_position)

    def get_semaphore_agent(self, position):
        agents_at_position = self.model.grid.get_cell_list_contents([position])
        for agent in agents_at_position:
            if isinstance(agent, TrafficLightAgent):
                return agent
        return None
    
    def is_semaphore_green(self, semaphore_agent, position):
        if semaphore_agent.state == "green":
            return True
        return False
            
    def check_agent(self, new_position):
        agents_at_position = self.model.grid.get_cell_list_contents([new_position])
        for agent in agents_at_position:
            if isinstance(agent, CarAgent):
                return False
        return True

    def move(self):
        # Get current position and allowed directions
        current_position = self.pos
        possible_current_directions = self.model.get_cell_directions(current_position)
        
        # Check for any available directions in this cell
        if not possible_current_directions:
            print(f"No directions available for agent at position {current_position}")
            return

        # Filter allowed directions and select one randomly
        possible_directions = self.get_possible_directions(possible_current_directions)
        if not possible_directions:
            print(f"No movement options for agent at position {current_position}")
            return
        
        # Checks if it can move acccording to the sempahore
        if not self.check_semaphore(current_position):
            return 

        direction = self.choose_direction(possible_directions)
        new_position = self.calculate_new_position(direction)

        if not self.check_agent(new_position):
            return
        
        self.update_position(new_position)

    def get_possible_directions(self, possible_current_directions):
        return [direction for direction, allowed in possible_current_directions.items() if allowed]

    def choose_direction(self, possible_directions):
        direction_weights = {
            "up": 1,
            "down": 1,
            "left": 1,
            "right": 1,
            "down_left": 0.1,
            "down_right": 0.1,
            "up_left": 0.1,
            "up_right": 0.1
        }
        weights = [direction_weights[direction] for direction in possible_directions]
        return random.choices(possible_directions, weights=weights, k=1)[0]
    
    def calculate_new_position(self, direction):
        direction_map = {
            "up": (0, 1),
            "down": (0, -1),
            "left": (-1, 0),
            "right": (1, 0),
            "down_left": (-1, -1),
            "down_right": (1, -1),
            "up_left": (-1, 1),
            "up_right": (1, 1),
        }
        dx, dy = direction_map[direction]
        return (self.pos[0] + dx, self.pos[1] + dy)
    
    def update_position(self, new_position):
        self.model.grid.move_agent(self, new_position)
        self.distance_travelled += 1
        self.pos = new_position


    def move_to_target(self):
        # Moverse hasta alcanzar el destino
        while self.active:
            if self.pos in self.parking_spots and self.distance_travelled > 0:
                self.active = False
                break
            else:
                moved = self.move()
                if not moved:  # Si no puede moverse, detener el bucle
                    break            
            
    def step(self):
        self.move_to_target()

## Traffic Light Agent

In [ ]:
class TrafficLightAgent(mesa.Agent):
    def __init__(self, traffic_light_id, state, model, group, position, monitored_positions):
        super().__init__(model)
        self.traffic_light_id = traffic_light_id
        self.state = state
        self.time_counter = 0
        self.group = group
        self.position = position
        self.monitored_positions = monitored_positions

    def set_state(self, new_state):
        """Change the state of the traffic light."""
        self.state = new_state

    def set_time_counter(self, new_time):
        """Set the time counter of the traffic light."""
        self.time_counter = new_time

    def cars_in_monitored_area(self):
        """Check if there are cars in the monitored positions."""
        in_area_cars = 0
        for pos in self.monitored_positions:
            agents = self.model.grid.get_cell_list_contents([pos])
            for agent in agents:
                if isinstance(agent, CarAgent):
                    in_area_cars += 1

        if(in_area_cars == 0):
            for pos in self.model.traffic_light_coords[self.traffic_light_id]["position"]:
                agents = self.model.grid.get_cell_list_contents([pos])
                for agent in agents:
                    if isinstance(agent, TrafficLightAgent):
                        agent.change_state("yellow")
        return in_area_cars
    
    def change_state(self, state):
        """Change the state of the traffic light automatically."""
        self.state = state

    def get_group_traffic_lights(self):
        """Get the traffic lights in the group."""
        return self.model.traffic_light_groups[self.group]

    def get_other_traffic_light_id(self):
        """Get the ID of the other traffic light in the same group."""
        for traffic_light_id in self.model.traffic_light_groups[self.group]:
            if traffic_light_id != self.traffic_light_id:
                return traffic_light_id
        return None
    
    def compare_group_traffic(self):
        other_traffic_light_id = self.get_other_traffic_light_id()
        #print(f"Agent Id: {self.traffic_light_id} | Other traffic light id: {other_traffic_light_id}")
        other_traffic_lights_positions = self.model.traffic_light_coords[other_traffic_light_id]["position"]
        self_traffic_light_positions = self.model.traffic_light_coords[self.traffic_light_id]["position"]
        
        other_traffic_lights = []
        self_traffic_lights = []
        for pos in other_traffic_lights_positions:
            agents = self.model.grid.get_cell_list_contents([pos])
            for agent in agents:
                if isinstance(agent, TrafficLightAgent):
                    #print(f"Agent Id: {self.traffic_light_id} | Other traffic_id: {agent.traffic_light_id} | This_id car count: {self.cars_in_monitored_area()} | Other traffic car count: {agent.cars_in_monitored_area()}")
                    other_traffic_lights.append(agent)
                    break
        
        for pos in self_traffic_light_positions:
            agents = self.model.grid.get_cell_list_contents([pos])
            for agent in agents:
                if isinstance(agent, TrafficLightAgent):
                    #print(f"Agent Id: {self.traffic_light_id} | Other traffic_id: {agent.traffic_light_id} | This_id car count: {self.cars_in_monitored_area()} | Other traffic car count: {agent.cars_in_monitored_area()}")
                    self_traffic_lights.append(agent)
                    break

        
        other_traffic_light_cars = other_traffic_lights[0].cars_in_monitored_area()
        self_traffic_light_cars = self_traffic_lights[0].cars_in_monitored_area()

        cars_in_monitored_area = self.cars_in_monitored_area()
        if cars_in_monitored_area == 0:
            for traffic_light_agent in self_traffic_lights:
                #Change the state of the other traffic light to red
                traffic_light_agent.change_state("yellow")
        elif cars_in_monitored_area > other_traffic_light_cars:
            #print(f"There are more cars in the monitored area of traffic light {self.traffic_light_id}({self.cars_in_monitored_area()} cars) than in the monitored area of traffic light {other_traffic_light_id}({other_traffic_light_cars} cars)")
            for traffic_light_agent in other_traffic_lights:
                #Change the state of the other traffic light to red
                traffic_light_agent.change_state("red")
                self.change_state("green")
                self.time_counter = 0

        
            #print(f"Traffic light {self.traffic_light_id}:{self.position} changed to green and traffic light {other_traffic_light_id} changed to red")
             
        

    def step(self):
        self.time_counter += 1
        self.compare_group_traffic()

        


## Model Deployment 

In [11]:
# Crear una instancia de TrafficModel
model = TrafficModel(24, 24, 0, coords, buildings_coords, parking_spots, traffic_light_coords)

# Crear una matriz para contar los agentes de tipo CarAgent
agent_counts = np.zeros((model.grid.width, model.grid.height))

"""# Imprimir las coordenadas de los agentes
print("Coordenadas de los agentes:")
for agent in model.agents:  # Itera sobre todos los agentes en el modelo
    if isinstance(agent, CarAgent):  # Verifica si es un CarAgent
        print(f"Agente en posición: {agent.pos}")
"""

for i in range(15):  # Eje    cutar el modelo por 10 pasos
  model.step()

C:\Users\ricky\AppData\Local\Temp\ipykernel_13184\2736903646.py:52: FutureWarning: The new PropertyLayer and _PropertyGrid classes experimental. It may be changed or removed in any and all future releases, including patch releases.
We would love to hear what you think about this new feature. If you have any thoughts, share them with us here: https://github.com/projectmesa/mesa/discussions/1932
  buildingLayer = mesa.space.PropertyLayer("building", self.width, self.height, np.int64(0), np.int64)


# Visualization 1

In [12]:
#%pip install solara

In [13]:
import solara


Define the agent portrayal function

In [14]:
def agent_portrayal(agent):
    size = 20
    color = "tab:red"
    shape = "circle"
    
    # Verificar el tipo de agente
    if isinstance(agent, CarAgent):
        size = 50
        color = "tab:blue"
        shape = "circle"
    elif isinstance(agent, TrafficLightAgent):
        if agent.state == "red":
            color = "tab:red"
        elif agent.state == "green":
            color = "tab:green"
        elif agent.state == "yellow":
            color = "yellow"
            
    return {"size": size, "color": color, "shape": shape}



 Define the model parameters

In [15]:
model_params = {
    "num_agents": {
        "type": "SliderInt",
        "value": 10,
        "label": "Number of agents:",
    },
    "width": 24,
    "height": 24,
}

Create the initial model instance

In [16]:
initial_model = TrafficModel(
    num_agents=2,
    width=24,
    height=24,
    coords = coords,
    buildings_coords=buildings_coords,
    parking_coords=parking_spots,
    traffic_light_coords=traffic_light_coords
)



Spawn: ((5, 17)), Target: ((10, 7))
Spawn: ((3, 6)), Target: ((4, 3))


Create the visualization components

In [17]:
propertylayer_portrayal={"building": {"color":"blue","colorbar":False}, "parking":{"color":"yellow","colorbar":False}, "traffic_monitoring":{"color":"gray","colorbar":False}}
SpaceGraph = make_space_component(agent_portrayal, propertylayer_portrayal=propertylayer_portrayal)

Create the SolaraViz page

In [18]:
page = SolaraViz(
    initial_model,
    components=[SpaceGraph],
    model_params=model_params,
    name="Traffic Simulation",
)
page

d:\Aplicaciones\Python\Lib\site-packages\mesa\visualization\mpl_space_drawing.py:272: UserWarning: the following fields are not used in agent portrayal and thus ignored: shape.
  arguments = collect_agent_data(space, agent_portrayal, size=s_default)
d:\Aplicaciones\Python\Lib\site-packages\mesa\visualization\mpl_space_drawing.py:272: UserWarning: the following fields are not used in agent portrayal and thus ignored: shape.
  arguments = collect_agent_data(space, agent_portrayal, size=s_default)


Cannot show ipywidgets in text